In [1]:
import warnings
warnings.filterwarnings('ignore')
import keras
keras.__version__

Using TensorFlow backend.


'2.2.0'


다음 코드는 기본적인 컨브넷의 모습입니다. **Conv2D**와 **MaxPooling2D** 층을 쌓아 올렸습니다. 


컨브넷이 **(image_height, image_width, image_channels)** 크기의 입력 텐서를 사용한다는 점이 중요합니다(배치 차원은 포함하지 않습니다).

이 예제에서는 **MNIST** 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하도록 컨브넷을 설정해야 합니다.

In [2]:
from keras import layers
from keras import models

In [5]:
model = models.Sequential()
'''
# Arguments
    filters: Integer, the dimensionality of the output space
       
    kernel_size: 
    strides: default (1,1)
    padding: "valid"` or `"same"
    activation:
    ...

# Input shape
    4D tensor with shape:
    `(samples, channels, rows, cols)`
    if `data_format` is `"channels_first"`
    or 4D tensor with shape:
    `(samples, rows, cols, channels)`
    if `data_format` is `"channels_last"`.

# Output shape
    4D tensor with shape:
    `(samples, filters, new_rows, new_cols)`
    if `data_format` is `"channels_first"`
    or 4D tensor with shape:
    `(samples, new_rows, new_cols, filters)`
    if `data_format` is `"channels_last"`.
    `rows` and `cols` values might have changed due to padding.
    
    
https://keras.io/layers/convolutional/#zeropadding2d    
'''

model.add(layers.Conv2D(32, (3, 3),activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

위에서 만든 컨브넷 구조를 출력

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서입니다. 높이와 넓이 차원은 네트워크가 깊어질수록 작아지는 경향이 있습니다. 채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절됩니다(32개 또는 64개).

다음 단계에서 마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 완전 연결 네트워크에 주입합니다. 이 네트워크는 이미 익숙하게 보았던 Dense 층을 쌓은 분류기입니다. 이 분류기는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서입니다. 그래서 먼저 3D 출력을 1D 텐서로 펼쳐야 합니다. 그다음 몇 개의 Dense 층을 추가합니다:

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용합니다. 이제 전체 네트워크는 다음과 같습니다:

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

여기에서 볼 수 있듯이 (3, 3, 64) 출력이 (576,) 크기의 벡터로 펼쳐진 후 Dense 층으로 주입되었습니다.

이제 MNIST 숫자 이미지에 이 컨브넷을 훈련합니다.

In [9]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [10]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 57s 954us/step - loss: 0.1759 - acc: 0.9449
Epoch 2/5
60000/60000 [==============================] - 56s 929us/step - loss: 0.0472 - acc: 0.9856
Epoch 3/5
60000/60000 [==============================] - 58s 959us/step - loss: 0.0331 - acc: 0.9899
Epoch 4/5
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0250 - acc: 0.9924
Epoch 5/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0194 - acc: 0.9942


테스트 데이터에서 모델 평가

In [11]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 3s 288us/step


0.9906